# Day 7 - Lab 1: Advanced Agent Workflows with MCP for New Hire Onboarding

**Objective:** Master the Model Context Protocol (MCP) to build reliable, predictable agents for automating new employee onboarding workflows.

**Estimated Time:** 135 minutes

## Introduction

Welcome to Day 7! Today, you'll learn how the Model Context Protocol (MCP) transforms unreliable AI agents into production-ready systems. You'll build an onboarding automation agent that demonstrates why MCP is critical for enterprise AI applications.

### The Problem with Unstructured Context

When onboarding new employees, HR teams juggle multiple data sources:
- Policy documents (formal requirements)
- Slack messages (informal requests)
- Access matrices (system permissions)
- Training catalogs (learning requirements)

Without structure, AI agents make inconsistent decisions. MCP solves this by providing:
1. **Resources**: Structured, versioned data sources
2. **Tools**: Validated, idempotent operations
3. **Schema Enforcement**: Type-safe interactions

By the end of this lab, you'll build an MCP-powered onboarding agent that makes deterministic, auditable decisions.

## Step 1: Setup

We will install the necessary libraries for this lab, including `model-context-protocol` for creating structured prompts and `langchain-mcp-adapters` for easily integrating MCP with our LangChain agents.

**Model Selection:**
For tasks involving structured data and code, models with strong reasoning and instruction-following capabilities are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('model_context_protocol')
install_if_missing('langchain_mcp_adapters')

from utils import setup_llm_client, get_completion
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## Step 2: The Challenges

### Challenge 1 (Foundational): Manually Formatting an MCP Prompt

**Task:** Before using the SDK, manually write a prompt string that follows the MCP XML-style format.

> **Tip:** Notice the XML-style tags. This structure isn't just for looks; it gives the LLM clear signals about the role of each piece of information. `<request>` is the user's goal, while `<context>` provides the data needed to achieve it.

**Instructions:**
1.  Create a multi-line f-string for your prompt.
2.  The prompt should include the following MCP tags:
    * `<request>`: The user's high-level request (e.g., "Please refactor this code.").
    * `<context>`: A container for contextual information.
    * `<code>`: Inside `<context>`, place a snippet of Python code that needs refactoring.
    * `<instructions>`: Inside `<context>`, provide specific instructions for the refactoring (e.g., "Make this function more readable and add type hints.").
3.  Send this manually formatted string to the LLM and observe the output.

**Expected Quality:** A successful response from the LLM, demonstrating that it can understand and act upon the structured MCP format even without the SDK.

In [ ]:
code_to_refactor = "def my_func(a,b):\n  x=a+b\n  y=x*2\n  return y"

# TODO: Create a prompt string that manually uses MCP tags.
manual_mcp_prompt = f"""
# Your prompt here
"""

print("--- Sending Manually Formatted MCP Prompt ---")
response = get_completion(manual_mcp_prompt, client, model_name, api_provider)
print(response)

### Challenge 2 (Intermediate): Using the MCP SDK to Build a Prompt

**Task:** Now, use the `model-context-protocol` SDK to programmatically build the same structured prompt.

**Instructions:**
1.  Import `Request`, `Context`, `Code`, and `Instructions` from the `model_context_protocol` library.
2.  Create an instance of the `Code` context item, passing the `code_to_refactor`.
3.  Create an instance of the `Instructions` context item.
4.  Create a `Context` object, passing a list containing your `Code` and `Instructions` items.
5.  Create the final `Request` object, passing the user's request text and the `Context` object.
6.  Use the `.render()` method on the `Request` object to get the formatted string and print it to verify it matches your manual prompt.

**Expected Quality:** A Python script that programmatically generates a valid MCP-formatted string, demonstrating a more robust and maintainable way to construct complex prompts.

In [ ]:
from model_context_protocol import Request, Context, Code, Instructions

# TODO: Use the MCP SDK to build the request programmatically.

# 1. Create Code and Instructions items
code_item = None # Your code here
instructions_item = None # Your code here

# 2. Create the Context object
context_obj = None # Your code here

# 3. Create the final Request object
mcp_request = None # Your code here

# 4. Render the request to a string
rendered_prompt = mcp_request.render()

print("--- Programmatically Rendered MCP Prompt ---")
print(rendered_prompt)

### Challenge 3 (Advanced): Building a Context-Aware Refactoring Agent

**Task:** Use the `langchain-mcp-adapters` library to create a LangChain agent that automatically structures its inputs using MCP.

> **What do the adapters do?** The `langchain-mcp-adapters` library acts as a bridge. It lets you define the high-level structure of your context (e.g., 'I need code and instructions') and automatically handles the work of formatting it into the proper MCP string for the LLM.

**Instructions:**
1.  Import `McpChatPromptTemplate` from `langchain_mcp_adapters`.
2.  Create a list of context builders. For this lab, you'll need one for `Code` and one for `Instructions`. The adapter library provides these.
3.  Create an instance of `McpChatPromptTemplate`, passing in your list of context builders and a request template string.
4.  Create a simple LangChain chain by piping this special prompt template to your LLM.
5.  Invoke the chain. The input should be a dictionary containing keys that match your context builders (e.g., `code` and `instructions`) and your request template variables.
6.  The adapter will automatically build the full MCP-formatted prompt before sending it to the LLM.

**Expected Quality:** A functioning LangChain agent that seamlessly and automatically uses the Model Context Protocol, demonstrating how to build robust, production-ready agents that handle complex, multi-part context.

In [ ]:
from langchain_mcp_adapters import McpChatPromptTemplate
from langchain_mcp_adapters.context_builders import CodeContextBuilder, InstructionsContextBuilder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model_name)

# TODO: 1. Create a list of context builders
context_builders = [] # Your list here

# TODO: 2. Create the McpChatPromptTemplate
mcp_prompt_template = None # Your prompt template here

# TODO: 3. Create the LangChain agent chain
refactoring_agent = None # Your chain here

# TODO: 4. Invoke the agent with a dictionary of inputs
refactoring_input = {
    "user_request": "Please refactor this code to be more Pythonic.",
    "code": "def f(data):\n  r = []\n  for i in data:\n    if i % 2 == 0:\n      r.append(i*i)\n  return r",
    "instructions": "Use a list comprehension and add type hints."
}

print("--- Invoking MCP-powered Refactoring Agent ---")
refactored_code = refactoring_agent.invoke(refactoring_input)
print(refactored_code)

## Lab Conclusion

Excellent work! You have learned how to use the Model Context Protocol to create structured, reliable prompts for your AI agents. You progressed from manually writing MCP-formatted strings to using the SDK for programmatic construction, and finally to using the LangChain adapter for seamless integration. This skill is crucial for building advanced agents that need to handle diverse and complex contextual information in a predictable way.

> **Key Takeaway:** Structuring prompts with a clear protocol like MCP makes agents more reliable. It separates the user's *request* from the *context* the agent needs, reducing ambiguity and allowing you to build more predictable, production-ready AI systems.